In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

import json
from time import time
import locale
import re
import numpy as np




import requests
import json
import pandas as pd
import numpy as np



In [3]:
files_path = "files"
TQA_files_path = os.path.join(files_path, "TruthfulQA")
output_files = "out_files_temp"

# Manual Review

Create the reduced dataset from TruthfulQA for the manual review

In [5]:
import copy, csv, jsonlines
def getList(input):
  sentences = input.split(';')
  sentences = [sentence.strip() for sentence in sentences]
  return sentences

with open(os.path.join(TQA_files_path, "truthful_qa.csv"), "r", encoding="utf-8") as csvfile:
  reader = csv.DictReader(csvfile)
  with jsonlines.open(os.path.join(output_files, "reduced_for_review_truthful_qa.jsonl"), "w") as writer:
    for line in reader:

      new_data = dict()
      new_data["Type"] = line["\ufeffType"]
      new_data["Category"] = line["Category"]
      new_data["Question"] = line["Question"]
      new_data["Source"] = line["Source"]

      #get only the first two answers (whenever possible)
      c_answers = getList(line["Correct Answers"])
      count = 0
      for answer in c_answers:
        if count != 2:
          final_data = copy.deepcopy(new_data)
          final_data["Answer"] = answer
          final_data["Real_label"] = True
          writer.write(final_data)
          count = count + 1
        else:
          break

      #get only the first two answers (whenever possible)
      i_answers = getList(line["Incorrect Answers"])
      count = 0
      for answer in i_answers:
        if count != 2:
          final_data = copy.deepcopy(new_data)
          final_data["Answer"] = answer
          final_data["Real_label"] = False
          writer.write(final_data)
          count = count + 1
        else:
          break


Add the ids from the original and complete TruthfulQA dataset (see the other notebook for the original file creation)

In [8]:
df1 = pd.read_json(os.path.join(TQA_files_path, 'WITH_EV_truthful_qa_UNROLLED.jsonl'), lines = True)
df2 = pd.read_json(os.path.join(output_files, "reduced_for_review_truthful_qa.jsonl"), lines = True)
# Create a mapping dictionary from df1's question-answer pairs to IDs
mapping = df1.groupby(['Question', 'Answer'])['ID'].max().to_dict()
# Apply the mapping to df2
df2['ID'] = df2.apply(lambda row: mapping[(row['Question'], row['Answer'])], axis=1)
df2 = df2.rename(columns={'Real_label': 'Factuality_ground_label'})
#reorder the columns
desired_columns = ['ID', 'Type', 'Category', 'Question', 'Answer', 'Factuality_ground_label', 'Source']
df2 = df2[desired_columns]
df2.to_json(os.path.join(output_files, "reduced_for_review_truthful_qa.jsonl"), orient='records', lines=True)


Test prompts on manually reviewed dataset


In [24]:
#LONGPROMPT
from sklearn.metrics import accuracy_score

os.chdir('/content/drive/MyDrive/Tesi/Codice/FINAL/src')
!python qa_relevance.py --model 'Starling-LM-7B-alpha' --input 'manual_checkworthiness_dataset.jsonl' --output 'longprompt_checkworthiness_dataset.jsonl' --prompt 'checkworthiness_LongPrompt.txt'--resume
os.chdir('/content/drive/MyDrive/Tesi/Codice/FINAL/files/checkworthiness_detection')
df = pd.read_json("longprompt_checkworthiness_dataset.jsonl", lines = True)
manual = df['manual_checkworthy'].astype(int)
model = df['model_checkworthiness'].astype(int)
print(f'accuracy: {accuracy_score(manual, model):.4f}')



accuracy: 0.7653


In [26]:
#SHORTPROMPT
from sklearn.metrics import accuracy_score

os.chdir('/content/drive/MyDrive/Tesi/Codice/FINAL/src')
!python qa_relevance.py --model 'Starling-LM-7B-alpha' --input 'manual_checkworthiness_dataset.jsonl' --output 'shortprompt_checkworthiness_dataset.jsonl' --prompt 'checkworthiness_ShortPrompt.txt'--resume
os.chdir('/content/drive/MyDrive/Tesi/Codice/FINAL/files/checkworthiness_detection')
df = pd.read_json("shortprompt_checkworthiness_dataset.jsonl", lines = True)
manual = df['manual_checkworthy'].astype(int)
model = df['model_checkworthiness'].astype(int)
print(f'accuracy: {accuracy_score(manual, model):.4f}')

accuracy: 0.8367


#Automatic Checkworthiness detection

Filter the problematic samples (Disputable categories, "nothing, many, few")

In [ ]:
os.chdir('/content/drive/MyDrive/Tesi/Codice/FINAL/files')
df = pd.read_json("truthful_qa_UNROLLED.jsonl", lines = True)

In [ ]:
categories_to_exclude = ['Myths and Fairytales', 'Paranormal', 'Superstitions', 'Religion', 'Fiction', 'Advertising', 'Indexical Error: Identity']
# Define words to exclude
words_to_exclude = ["many", "few", "nothing", "ultimate"]

# Create a regex pattern to match any of the words
pattern = '|'.join(words_to_exclude)

# Filter DataFrame to exclude rows where 'Answer' contains any of the specified words (case insensitive)
filtered_df = df[~df['Answer'].str.contains(pattern, case=False, regex=True) & ~df['Category'].isin(categories_to_exclude)]

In [ ]:
filtered_df.to_json('FILTERED_truthful_qa_UNROLLED.jsonl', orient='records', lines=True)

Create Checkworthiness dataset (ShortPrompt)

In [ ]:
os.chdir('/content/drive/MyDrive/Tesi/Codice/FINAL/files')
filtered_df = pd.read_json("FILTERED_truthful_qa_UNROLLED.jsonl", lines = True)

In [ ]:
os.chdir('/content/drive/MyDrive/Tesi/Codice/FINAL/src')
!python qa_relevance.py --model 'Starling-LM-7B-alpha' --input 'FILTERED_truthful_qa_UNROLLED.jsonl' --output 'or_model_checkworthiness_dataset.jsonl' --prompt 'checkworthiness_ShortPrompt.txt' --resume

Modify the already reviewed samples with the labels manually obtained

In [2]:
import pandas as pd

os.chdir('/content/drive/MyDrive/Tesi/Codice/FINAL/files/checkworthiness_detection')
df1 = pd.read_json("or_model_checkworthiness_dataset.jsonl", lines = True)
df2 = pd.read_json("manual_checkworthiness_dataset.jsonl", lines = True)
df2 = df2[["ID", "manual_checkworthy"]]
# Merge DataFrames on 'ID'
merged_df = pd.merge(df1, df2, on='ID', how='left')

# Update 'model_label' with 'manual_label' where applicable
merged_df['Model_checkworthiness_label'] = merged_df['manual_checkworthy'].combine_first(merged_df['Model_checkworthiness_label'])

# Drop the 'manual_label' column as it's no longer needed
updated_df1 = merged_df.drop(columns=['manual_checkworthy'])

#updated_df1.to_json('model_checkworthiness_dataset.jsonl', orient='records', lines=True)

Select only the checkworthy samples

In [6]:
os.chdir('/content/drive/MyDrive/Tesi/Codice/FINAL/files/checkworthiness_detection')
df1 = pd.read_json("model_checkworthiness_dataset.jsonl", lines = True)
check_df = df1[df1['Model_checkworthiness_label'] == True]
check_df.to_json('checkworthy_samples_dataset.jsonl', orient='records', lines=True)
len(check_df)

3944